# Topic Modeling 
This notebook aims to take as input the texts who have been processed and use it to find the most relevants topics and the words that are relevant for the sentimental analysis.

**Implementation**
- TF-IDF
- FinBERT
- LSA 

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import glob
import re
import pandas as pd
import import_ipynb
import os
from tqdm.notebook import tqdm

from pre_processing import processing

#from finbert_embedding.embedding import FinbertEmbedding


importing Jupyter notebook from pre_processing.ipynb


[nltk_data] Downloading package stopwords to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Roger
[nltk_data]     Gualberti\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['4427353', '4442143', '4467506', '4508456', '4508457', '4508458', '4508459', '4508460', '4508462', '4508464', '4508465', '4508466', '4508467', '4508468', '4508470', '4508471', '4508472', '4508473', '4508474', '4508475', '4508476', '4508477', '4508478', '4508479', '4508480', '4508481', '4508482', '4508483', '4508484', '4508486', '4508487', '4508488', '4508489', '4508490', '4508491', '4508492', '4508493', '4508494', '4508495', '4508497', '4508498', '4508499', '4508500', '4508501', '4508502', '4508503', '4508504', '4508505', '4508506', '4508507', '4508508', '4508509', '4508510', '4508511', '4508512', '4508513', '4508514', '4508515', '4508516', '4508517', '4508518', '4508519', '4508520', '4508521', '4508522', '4508523', '4508524', '4508525', '4508526', '4508527', '4508528', '4508529', '4508530', '4508531', '4508532', '4508533', '4508534', '4508535', '4508536', '4508537', '4508538', '4508539', '4508540', '4508541', '4508542', '4508543', '4508544', '4508545', '4508546', '4508547', '4508548'

Number of articles 1161
Number of words before removing the stop words 0
Number of words after removing the stop words 0


  0%|          | 0/1161 [00:00<?, ?it/s]

### Import the text and process it 

In [2]:
#list_articles = glob.glob("data/earning_call/*")
dir_path = "data/earning_call/"
list_articles = os.listdir(dir_path) 
#print(list_articles[:5])
texts = []
first_sentence = []
articles = []
for s in list_articles:
    with open(dir_path + s, encoding = 'ISO-8859-1') as f:
        x = int(re.sub('data/earning_call/','',s))
        articles.append(x)
        t = f.read()
        texts.append(t)
print('Number of articles', len(texts))


Number of articles 1161


In [3]:
texts = [processing(x) for x in tqdm(texts)]

  0%|          | 0/1161 [00:00<?, ?it/s]

### TF-IDF
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [4]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(texts)
X_tfidf.shape

(1161, 42900)

In [5]:
dict_w_index = vectorizer.vocabulary_
dict_index_w = {v: k for k, v in dict_w_index.items()}

Take the top n words depending on the score with the TF-IDF

In [6]:
n =10
top_n = []
for i in range(X_tfidf.shape[0]):
    index = X_tfidf[i,].nonzero()[1]
    words_of_index = [dict_index_w[x] for x in index]
    score_of_index = [X_tfidf[i,x] for x in index]
    x = list(zip(words_of_index,score_of_index))
    x.sort(key=lambda x: -x[1])
    a = [w[0] for w in x[:n]]
    top_n.append(a)

In [7]:
for i in range(len(list_articles)):
    list_articles[i] = dir_path+list_articles[i]
df = pd.DataFrame({'article':articles,'file_path':list_articles,'top_n_words':top_n})
df.to_pickle("./top_n_words_tfidf.pkl")  

In [8]:
df

,article,file_path,top_n_words
0,4427353,data/earning_call/4427353,"[sales, gupta, fy, rogue, san, ice, automotive..."
1,4442143,data/earning_call/4442143,"[makoto, shimbun, kamizawa, asayama, alliance,..."
2,4467506,data/earning_call/4467506,"[hamaguchi, sadayuki, asayama, hans, ashwani, ..."
3,4508456,data/earning_call/4508456,"[thiago, isabella, derivatives, grain, mariano..."
4,4508457,data/earning_call/4508457,"[dariusz, ivy, dta, communities, southwest, ja..."
...,...,...,...
1156,4511738,data/earning_call/4511738,"[mister, simeon, members, washing, cull, store..."
1157,4511741,data/earning_call/4511741,"[gandhi, prithvi, users, jensen, machining, cu..."
1158,4511742,data/earning_call/4511742,"[payers, sera, preterm, anita, dustin, multipl..."
1159,4511743,data/earning_call/4511743,"[bronchiectasis, kathleen, skarvan, airway, sa..."


### FinBERT 
https://pypi.org/project/finbert-embedding/

In [ ]:
finbert = FinbertEmbedding()

In [ ]:
X_FinB = np.zeros((len(texts),768))
k=0
for text in texts:
    X_FinB[k,] = finbert.sentence_vector(text)
    k+=1

In [ ]:
X_FinB.shape